# Choosing between multiple tools
- Quick Start 에서는 Chain을 구성하고 하나의 툴을 호출해보았습니다.
- 본 세션에서는 체인을 어떻게 확장하여 여러 도구 중에서 선택할 수 있게 할지에 대해 알아보겠습니다.


# tool 정의 방식 참조
![Agent_Tool_between_tools](imgs/Agent_tool_Choosing_between_tools.png)

In [1]:
from dotenv import load_dotenv
load_dotenv('../dot.env')

True

In [2]:
# Tools
from langchain_core.tools import tool

# 덧셈, 곱셈, 지수 연산 툴 정의
@tool
def multiply(first_int: int, second_int: int) -> int:
    """Multiply two integers together."""
    return first_int * second_int

@tool
def add(first_int: int, second_int: int) -> int:
    "Add two integers."
    return first_int + second_int


@tool
def exponentiate(base: int, exponent: int) -> int:
    "Exponentiate the base to the exponent power."
    return base**exponent

In [3]:
# LLM은 gpt-3.5-turbo를 사용함.
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model = "gpt-3.5-turbo")

In [6]:
from operator import itemgetter
from typing import Dict, List, Union
from langchain.prompts import PromptTemplate
from langchain_core.messages import AIMessage
from langchain_core.runnables import(
    RunnablePassthrough,
    RunnableLambda,
    Runnable,
    RunnableMap
)

tools = [multiply, add, exponentiate] # 도구 리스트 정의
llm_with_tools = llm.bind_tools(tools) # llm에 도구 바인딩

def call_tools(msg: AIMessage) -> Runnable:
    """
    이 함수는 AIMessage 객체를 받아서 각 도구 호출을 순차적으로 처리합니다.
    각 도구는 'tools' 리스트에 정의된 도구 객체를 사용하여 'tool_map' 사전에 이름을 키로 매핑합니다.
    이 매핑을 통해 각 도구 호출에 필요한 도구 함수를 식별하고 실행할 수 있습니다.
    
    Args:
    msg (AIMessage): 도구 호출 정보가 포함된 메시지 객체.

    Returns:
    Runnable: 도구 호출 결과를 포함하는 리스트.
    """
    # 도구 이름을 키로 하고 도구 객체를 값으로 하는 사전 생성
    tool_map = {tool.name : tool for tool in tools}
    # 메시지에서 도구 호출 정보 복사
    tool_calls = msg.tool_calls.copy()
    print(msg, '\n')
    print(msg.tool_calls)
    # 각 도구 호출에 대해
    for tool_call in tool_calls:
        # 도구 이름을 사용하여 해당 도구 객체를 찾고, 인자를 전달하여 도구 함수를 호출
        tool_call["output"] = tool_map[tool_call["name"]].invoke(tool_call["args"])
    # 도구 호출 결과 반환
    return tool_calls

chain = llm_with_tools | call_tools



- AIMessage: AIMessage는 인공지능과의 상호작용에서 메시지 객체를 정의하는 데 사용됩니다. 이 객체는 도구 호출 정보와 같은 데이터를 포함할 수 있습니다.
- Runnable: Runnable은 실행 가능한 작업을 추상화하는 인터페이스입니다. 이를 구현하는 객체는 특정 작업을 수행할 수 있습니다.
- RunnableLambda: RunnableLambda는 람다 함수를 사용하여 간단한 작업을 실행할 수 있는 Runnable의 구현입니다.
- RunnableMap: RunnableMap은 여러 Runnable 객체를 매핑하여 복합 작업을 수행할 수 있게 해주는 클래스입니다.
- RunnablePassthrough: RunnablePassthrough는 입력을 그대로 출력으로 전달하는 Runnable의 구현입니다. 이는 데이터 변환 없이 데이터를 다음 단계로 넘기는 데 사용됩니다.

In [8]:
chain.invoke('23의 7배는?')[0]

content='' additional_kwargs={'tool_calls': [{'id': 'call_gzmHTxLeZ6VdrnZL8jdA2OKW', 'function': {'arguments': '{"first_int":23,"second_int":7}', 'name': 'multiply'}, 'type': 'function'}]} response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 109, 'total_tokens': 128}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None} id='run-0f5aec37-493b-46c4-8815-2fbe45a09585-0' tool_calls=[{'name': 'multiply', 'args': {'first_int': 23, 'second_int': 7}, 'id': 'call_gzmHTxLeZ6VdrnZL8jdA2OKW'}] usage_metadata={'input_tokens': 109, 'output_tokens': 19, 'total_tokens': 128} 

[{'name': 'multiply', 'args': {'first_int': 23, 'second_int': 7}, 'id': 'call_gzmHTxLeZ6VdrnZL8jdA2OKW'}]


{'name': 'multiply',
 'args': {'first_int': 23, 'second_int': 7},
 'id': 'call_gzmHTxLeZ6VdrnZL8jdA2OKW',
 'output': 161}

In [ ]:
llm_with_tools